In [1]:
import numpy as np
import pandas as pd
import itertools

# dataPath =Data/carSales.csvs.csv"
dataPath ="Data/carSales_shangxuan.csv"


def edit_col_type(df, num_col, str_col):
    int_col = df.select_dtypes(include=np.int64).columns.tolist()
#     print(int_col)
    for col in int_col:
        uni_val = df[col].unique()
        is_year = True
        for val in uni_val:
            if val not in range(1900, 2030):
                is_year = False
#                 print('False, val is ', val)
                break
        if is_year:
            num_col.remove(col)
            str_col.append(col)
    return num_col, str_col

df = pd.read_csv(dataPath)
df = df.fillna(0)
num_col = df.select_dtypes(include=np.number).columns.tolist()
str_col = [str for str in df.columns.tolist() if str not in num_col]
# dateCol = df.select_dtypes(include='datetime').columns.tolist()
# How to find date time column?


print("String Column: ", str_col)
print("Numeric Column: ", num_col)
num_col, str_col = edit_col_type(df, num_col, str_col)
print("String Column after editing: ", str_col)
print("Numeric Column after editing: ", num_col)

String Column:  ['Year', 'Brand', 'Category']
Numeric Column:  ['Sales']
String Column after editing:  ['Year', 'Brand', 'Category']
Numeric Column after editing:  ['Sales']


In [2]:
def groupby_func(func, df):
    if func == 'average':
        return df.groupby(list(subset), as_index=False).mean()
    elif func == 'sum':
        return df.groupby(list(subset), as_index=False).sum()
    elif func == 'max':
        return df.groupby(list(subset), as_index=False).max()
    elif func == 'min':
        return df.groupby(list(subset), as_index=False).min()
    elif func == 'median':
        return df.groupby(list(subset), as_index=False).median()


In [3]:
for subset in itertools.permutations(str_col, 1):
    print("")
    print("")
    print("---------------------------------------------------")
    print("Calculate max for the combination:", list(subset))
    func_list = ['average', 'sum', 'max', 'min', 'median']

    for num in num_col:
        for func in func_list:
            dff = groupby_func(func, df)
            idx = dff[num].idxmax()
            print('{} <{}> has the max <{}> <{}>, which is {}'.format(subset[0], dff.at[idx, subset[0]], func, num, dff.at[idx, num]))



---------------------------------------------------
Calculate max for the combination: ['Year']
Year <2007/1/1> has the max <average> <Sales>, which is 91726.56363636363
Year <2007/1/1> has the max <sum> <Sales>, which is 5044961
Year <2007/1/1> has the max <max> <Sales>, which is 473108
Year <2009/1/1> has the max <min> <Sales>, which is 2261
Year <2007/1/1> has the max <median> <Sales>, which is 57575


---------------------------------------------------
Calculate max for the combination: ['Brand']
Brand <Honda> has the max <average> <Sales>, which is 177257.04
Brand <Ford> has the max <sum> <Sales>, which is 5489641
Brand <Toyota> has the max <max> <Sales>, which is 473108
Brand <Ford> has the max <min> <Sales>, which is 21273
Brand <Honda> has the max <median> <Sales>, which is 203714


---------------------------------------------------
Calculate max for the combination: ['Category']
Category <Midsize> has the max <average> <Sales>, which is 149148.97435897434
Category <SUV> has

In [5]:
for subset in itertools.permutations(str_col, 2):
    print("")
    print("")
    print("---------------------------------------------------")
    print("Calculate max for the combination: ", list(subset))
    dff = df.groupby(list(subset), as_index=False).mean()
    for num in num_col:
        print()
        print("Calculate max for", num)
        grouped = dff.loc[dff.groupby(subset[0])[num].idxmax()]
        res = grouped[[subset[0], subset[1], num]]
        for index, row in res.iterrows():
            print('The max avg <{}> of <{} {}> is {} with <{}> equals {}'.format(num, subset[0], row[subset[0]], row[num], subset[1], row[subset[1]]))



---------------------------------------------------
Calculate max for the combination:  ['Year', 'Brand']

Calculate max for Sales
The max avg <Sales> of <Year 2007/1/1> is 206827.2 with <Brand> equals Honda
The max avg <Sales> of <Year 2008/1/1> is 203119.6 with <Brand> equals Honda
The max avg <Sales> of <Year 2009/1/1> is 164125.4 with <Brand> equals Honda
The max avg <Sales> of <Year 2010/1/1> is 163012.6 with <Brand> equals Honda
The max avg <Sales> of <Year 2011/1/1> is 149200.4 with <Brand> equals Honda


---------------------------------------------------
Calculate max for the combination:  ['Year', 'Category']

Calculate max for Sales
The max avg <Sales> of <Year 2007/1/1> is 179617.42857142858 with <Category> equals Midsize
The max avg <Sales> of <Year 2008/1/1> is 150622.0 with <Category> equals Midsize
The max avg <Sales> of <Year 2009/1/1> is 131875.0 with <Category> equals Midsize
The max avg <Sales> of <Year 2010/1/1> is 142726.25 with <Category> equals Midsize
The max

## Extract Top 1 Insight using QuickInsight Format

In [14]:
dim_range = dict()
for dim in str_col:
    dim_range[dim] = []
    dim_range[dim].extend(df[dim].unique())
print(dim_range)

{'Year': ['2007/1/1', '2008/1/1', '2009/1/1', '2010/1/1', '2011/1/1'], 'Brand': ['BMW', 'Ford', 'GMC', 'Honda', 'Hyundai', 'Mazda', 'Toyota', 'Volkswagen'], 'Category': ['Compact', 'Fullsize', 'Midsize', 'Sporty', 'Subcompact', 'SUV', 'Pickup', 'MPV']}


In [65]:
for subspaceLen in range(0, len(str_col)):
    for dims in itertools.combinations(str_col, subspaceLen):
#       dims: subspace中的元素
        breakdowns = [x for x in str_col if (x not in dims)]
#         print(breakdowns)
        space_range = []
        for dim in dims:
            space_range.append(dim_range[dim])
        combi = list(itertools.product(*space_range))
        for subspace in combi:
            for measure in num_col:
                if (len(dims)>0):
                    dff = df.loc[np.all([df[dims[i]] == subspace[i] for i in range(0, subspaceLen)], axis=0)]
                else:
                    dff =df
                for bd in breakdowns:
                    df_group = dff.groupby(bd, as_index=False).mean()
                    if not df_group.empty:
                        idx = df_group[measure].idxmax()
                        res_str = ';'.join(map(str,subspace) )
#                        CSV Result
                        print('{},{},{},{},{}'.format(res_str, bd, measure, df_group.at[idx, measure], df_group.at[idx, bd]))

,Year,Sales,91726.56363636363,2007/1/1
,Brand,Sales,177257.04,Honda
,Category,Sales,149148.97435897434,Midsize
2007/1/1,Brand,Sales,206827.2,Honda
2007/1/1,Category,Sales,179617.42857142858,Midsize
2008/1/1,Brand,Sales,203119.6,Honda
2008/1/1,Category,Sales,150622.0,Midsize
2009/1/1,Brand,Sales,164125.4,Honda
2009/1/1,Category,Sales,131875.0,Midsize
2010/1/1,Brand,Sales,163012.6,Honda
2010/1/1,Category,Sales,142726.25,Midsize
2011/1/1,Brand,Sales,149200.4,Honda
2011/1/1,Category,Sales,144712.75,Midsize
BMW,Year,Sales,34151.22222222222,2007/1/1
BMW,Category,Sales,99222.83333333333,Compact
Ford,Year,Sales,117825.18181818182,2011/1/1
Ford,Category,Sales,189015.6,Midsize
GMC,Year,Sales,70776.66666666667,2007/1/1
GMC,Category,Sales,83114.4,Pickup
Honda,Year,Sales,206827.2,2007/1/1
Honda,Category,Sales,314133.4,Midsize
Hyundai,Year,Sales,117909.2,2011/1/1
Hyundai,Category,Sales,161107.4,Midsize
Mazda,Year,Sales,47051.4,2007/1/1
Mazda,Category,Sales,107096.8,Compact
Toyota,Year,Sales,188699.4